![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FRetrieval&file=Retrieval+-+Firestore.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Firestore.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FRetrieval%2FRetrieval%2520-%2520Firestore.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Firestore.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Firestore.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Retrieval - Firestore
<p style="font-size: 45px;">IN PROGRESS - NOT COMPLETE</p>

In prior workflows, a series of documents was [processed into chunks](../Chunking/readme.md), and for each chunk, [embeddings](../Embeddings/readme.md) were created:

- Process: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb)
- Embed: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

Retrieving chunks for a query involves calculating the embedding for the query and then using similarity metrics to find relevant chunks. A thorough review of similarity matching can be found in [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb) - use dot product! As development moves from experiment to application, the process of storing and computing similarity is migrated to a [retrieval](./readme.md) system. This workflow is part of a [series of workflows exploring many retrieval systems](./readme.md).

**Firestore For Storage, Indexing, And Search**

[Firestore](https://cloud.google.com/firestore) is a fully managed, serverless document database on Google Cloud that scales automatically to meet any demand, without requiring partitioning or incurring downtime. It's ideal for mobile, web, and server development because it keeps data in sync across client apps with real-time listeners and offers offline support for mobile and web.

- **Data Model:**  Firestore stores data in documents (similar to JSON) comprised of key-value pairs.  Keys (fields) are mapped to values of various supported data types. ([see the full list here](https://cloud.google.com/firestore/docs/concepts/data-types?hl=en)) ([Learn more about the data model](https://cloud.google.com/firestore/docs/data-model?hl=en))
- **Flexible Structure:** Data is organized in a hierarchical structure where collections contain documents. Documents can be nested objects, and collections can have subcollections, providing flexibility in how you structure your data.
- **BigQuery Integration:**  Firestore integrates directly with BigQuery, allowing you to stream data into BigQuery or export query results to Firestore. ([Learn more about BigQuery integration](https://cloud.google.com/firestore/docs/solutions/bigquery?hl=en))
- **Generative AI Features:** Firestore offers integrated GenAI features, such as generating text embeddings and seamless integration with LangChain.
- **Vector Similarity Search:** Firestore provides built-in vector similarity search with indexing for efficient nearest neighbor matching. ([Learn more about vector search](https://cloud.google.com/firestore/docs/vector-search?hl=en))

**Use Case Data**

Buying a home usually involves borrowing money from a lending institution, typically through a mortgage secured by the home's value. But how do these institutions manage the risks associated with such large loans, and how are lending standards established?

In the United States, two government-sponsored enterprises (GSEs) play a vital role in the housing market:

- Federal National Mortgage Association ([Fannie Mae](https://www.fanniemae.com/))
- Federal Home Loan Mortgage Corporation ([Freddie Mac](https://www.freddiemac.com/))

These GSEs purchase mortgages from lenders, enabling those lenders to offer more loans. This process also allows Fannie Mae and Freddie Mac to set standards for mortgages, ensuring they are responsible and borrowers are more likely to repay them. This system makes homeownership more affordable and stabilizes the housing market by maintaining a steady flow of liquidity for lenders and keeping interest rates controlled.

However, navigating the complexities of these GSEs and their extensive servicing guides can be challenging.

**Approaches**

[This series](../readme.md) covers many generative AI workflows. These documents are used directly as long context for Gemini in the workflow [Long Context Retrieval With The Vertex AI Gemini API](../Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb). The workflow below uses a [retrieval](./readme.md) approach with the already generated chunks and embeddings.

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('google.cloud.firestore', 'google-cloud-firestore'),
    ('google.cloud.storage', 'google-cloud-storage')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable firestore.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [9]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [10]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'retrieval-firestore'

# GCS storage bucket name
GCS_BUCKET = PROJECT_ID

Packages

In [11]:
import os, json, time, glob, datetime

import numpy as np

# Vertex AI
from google.cloud import aiplatform
import vertexai.language_models # for embeddings API
import vertexai.generative_models # for Gemini Models
from vertexai.resources.preview import feature_store

# gcs client
from google.cloud import storage

# firestore
from google.cloud import firestore

In [12]:
aiplatform.__version__

'1.69.0'

Clients

In [13]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)

# gcs client
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

# firestore client
fs = firestore.Client(project = PROJECT_ID)

---
## Text & Embeddings For Examples

This repository contains a [section for document processing (chunking)](../Chunking/readme.md) that includes an example of processing mulitple large pdfs (over 1000 pages) into chunks: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb).  The chunks of text from that workflow are stored with this repository and loaded by another companion workflow that augments the chunks with text embeddings: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb).

The following code will load the version of the chunks that includes text embeddings and prepare it for a local example of retrival augmented generation.

### Get The Documents

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [14]:
local_dir = '../Embeddings/files/embeddings-api'

In [15]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Embeddings/files/embeddings-api', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../Embeddings/files/embeddings-api`


### Load The Chunks

In [16]:
jsonl_files = glob.glob(f"{local_dir}/large-files*.jsonl")
jsonl_files.sort()
jsonl_files

['../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0000.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0001.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0002.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0003.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0004.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0005.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0006.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0007.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0008.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0009.jsonl']

In [17]:
chunks = []
for file in jsonl_files:
    with open(file, 'r') as f:
        chunks.extend([json.loads(line) for line in f])
len(chunks)

9040

### Review A Chunk

In [18]:
chunks[0].keys()

dict_keys(['instance', 'predictions', 'status'])

In [19]:
chunks[0]['instance']['chunk_id']

'fannie_part_0_c17'

In [20]:
print(chunks[0]['instance']['content'])

# Selling Guide Fannie Mae Single Family

## Fannie Mae Copyright Notice

### Fannie Mae Copyright Notice

|-|
| Section B3-4.2, Verification of Depository Assets 402 |
| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |
| B3-4.2-02, Depository Accounts (12/14/2022) 405 |
| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |
| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |
| B3-4.2-05, Foreign Assets (05/04/2022) 411 |
| Section B3-4.3, Verification of Non-Depository Assets 412 |
| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |
| B3-4.3-02, Trust Accounts (04/01/2009) 413 |
| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |
| B3-4.3-04, Personal Gifts (09/06/2023) 415 |
| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |
| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |
| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423 |
| B3-4.3-08, Employer Assistance (09/29/2015) 423 |
| B3-4.3-09,

In [21]:
chunks[0]['predictions'][0]['embeddings']['values'][0:10]

[0.031277116388082504,
 0.03056905046105385,
 0.010865348391234875,
 0.0623614676296711,
 0.03228681534528732,
 0.05066155269742012,
 0.046544693410396576,
 0.05509665608406067,
 -0.014074751175940037,
 0.008380400016903877]

### Prepare Chunk Structure

Make a list of dictionaries with information for each chunk:

In [22]:
content_chunks = [
    dict(
        gse = chunk['instance']['gse'],
        chunk_id = chunk['instance']['chunk_id'],
        content = chunk['instance']['content'],
        embedding = chunk['predictions'][0]['embeddings']['values']
    ) for chunk in chunks
]

### Query Embedding

Create a query, or prompt, and get the embedding for it:

Connect to models for text embeddings. Learn more about the model API:
- [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

In [23]:
question = "Does a lender have to perform servicing functions directly?"

In [24]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')

In [25]:
question_embedding = embedder.get_embeddings([question])[0].values
question_embedding[0:10]

[-0.0005117303808219731,
 0.009651427157223225,
 0.01768726110458374,
 0.014538003131747246,
 -0.01829824410378933,
 0.027877431362867355,
 -0.021124685183167458,
 0.008830446749925613,
 -0.02669006586074829,
 0.06414774805307388]

---
## Retrieval With Firestore


Firestore structure:

database of which there is a `default` which includes a free qouta per month
https://cloud.google.com/firestore/docs/manage-databases#the_default_database
https://cloud.google.com/firestore/quotas#free-quota

collections are groups of doucments in a database - like a container or folder
https://cloud.google.com/firestore/docs/data-model#collections

and then the actual piece of data, the document which can have a nested structure itself
https://cloud.google.com/firestore/docs/data-model#documents




https://cloud.google.com/python/docs/reference/firestore/latest/google.cloud.firestore_v1.client.Client

### Prepare Data For Firestore

Basically, think dictionary or JSON, with `key:value` pairs and nesting is supported.  Read more about [data structure](https://cloud.google.com/firestore/docs/concepts/structure-data) and [supported data types](https://cloud.google.com/firestore/docs/concepts/data-types).

In [30]:
input_data = [
    dict(
        id = chunk['instance']['chunk_id'],
        embedding = chunk['predictions'][0]['embeddings']['values'],
        gse = chunk['instance']['gse'],
        content = chunk['instance']['content']
    ) for chunk in chunks
]

### Setup Firestore

If this is the first time using Firestore in the GCP project then the default database will need to be setup. Read about the steps [here](https://cloud.google.com/firestore/docs/create-database-server-client-library#create_a_in_native_mode_database).
- Go to the [Firestore Viewer](https://console.cloud.google.com/firestore/data)
- If a Database ID value of **(default)** is present then Firebase is already setup for this workflow, otherwise:
    - Select **Create Database**
        - In 'Select your Firestore mode' Choose **Native mode (recommended)** and then click **Continue**
        - In `Configure your database` leave the `Database ID` as **(default)** and then click **Create Database**
            - Location Type: Choose a Region or Multi-region or leave the defaulted values

### Create/Retrieve Collection

The client is setup above without a specific database reference which will defer to the `default` database.

In [35]:
collection = fs.collection(f'{SERIES}-{EXPERIMENT}')
doc_list = collection.limit(1).get()

In [36]:
if doc_list:
    print(f"Collection '{collection.id}' exists.")
else:
    print(f"Collection '{collection.id}' does not exist.")

Collection 'applied-genai-retrieval-firestore' exists.


### Add/Retrieve First Document In The Collection

From the collection object you can refer to a `.document('name')` object by name, even prior to creating it.  From this document object you can add its data to the database with `.set()`, update its contents with `.update` and remove the document with `.delete()`. 

In [37]:
document = collection.document(input_data[0]['id'])
if not doc_list:
    document = document.set(input_data[0])
else:
    document = document.get()

In [40]:
local_document = document.to_dict()
local_document.keys()

dict_keys(['embedding', 'id', 'gse', 'content'])

In [43]:
print(local_document['gse'])

fannie


In [42]:
print(local_document['content'])

# Selling Guide Fannie Mae Single Family

## Fannie Mae Copyright Notice

### Fannie Mae Copyright Notice

|-|
| Section B3-4.2, Verification of Depository Assets 402 |
| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |
| B3-4.2-02, Depository Accounts (12/14/2022) 405 |
| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |
| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |
| B3-4.2-05, Foreign Assets (05/04/2022) 411 |
| Section B3-4.3, Verification of Non-Depository Assets 412 |
| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |
| B3-4.3-02, Trust Accounts (04/01/2009) 413 |
| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |
| B3-4.3-04, Personal Gifts (09/06/2023) 415 |
| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |
| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |
| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423 |
| B3-4.3-08, Employer Assistance (09/29/2015) 423 |
| B3-4.3-09,

### Batch Add Documents To The Collection

If only the first document is loaded then load the rest as a [batch](https://cloud.google.com/firestore/docs/manage-data/transactions#batched-writes).

>Determining whether all documents have been loaded, as opposed to just the first, presents an interesting challenge. In a typical data source, you might perform a document/record count, but this operation is not feasible in Firestore, as it would necessitate reading all documents. If a total count is required, the entire collection could be streamed using `.stream()` and iterated over. However, a more efficient method is employed here: retrieving a maximum of two documents and checking if the result contains more than one. This approach is both fast and simple! Alternatively, a query could be made for the second document, and if it's missing, proceed to load it and any remaining documents.

Any batch can contains documents with `.set()`, `.update()`, and `.delete()` actions.

In [53]:
doc_list = collection.limit(2).get()
if len(doc_list) > 1:
    print(f"Documents already loaded to this collection '{collection.id}'.")
elif len(doc_list) == 1:
    print(f"Need to load the remaining documents...")
    
    # load remaining documents as a series of batch write operations
    batchsize = 100
    for b, i in enumerate(range(1, len(input_data), batchsize)):
        batch = fs.batch()
        for record in input_data[i:i+batchsize]:
            curr_doc = collection.document(record['id'])
            batch.set(curr_doc, record)
        batch.commit()
        print(f"Completed batch number {b+1} of {len(input_data[i:i+batchsize])} records")
    print(f"Documents loaded to the collection: '{collection.id}'")
    
else:
    print(f"Collection '{collection.id}' does not exist.")

Need to load the remaining documents...
Completed batch number 1 of 100 records
Completed batch number 2 of 100 records
Completed batch number 3 of 100 records
Completed batch number 4 of 100 records
Completed batch number 5 of 100 records
Completed batch number 6 of 100 records
Completed batch number 7 of 100 records
Completed batch number 8 of 100 records
Completed batch number 9 of 100 records
Completed batch number 10 of 100 records
Completed batch number 11 of 100 records
Completed batch number 12 of 100 records
Completed batch number 13 of 100 records
Completed batch number 14 of 100 records
Completed batch number 15 of 100 records
Completed batch number 16 of 100 records
Completed batch number 17 of 100 records
Completed batch number 18 of 100 records
Completed batch number 19 of 100 records
Completed batch number 20 of 100 records
Completed batch number 21 of 100 records
Completed batch number 22 of 100 records
Completed batch number 23 of 100 records
Completed batch number 24 

**NOTE:** An alternative to batch is using asychronous tasks - [read more here](https://cloud.google.com/firestore/docs/create-database-server-client-library#add_data).

### Retrieve Embedding For Entity

Retrieve an entry including embedding for a specific id (entity):

In [54]:
result = collection.document('fannie_part_0_c40').get().to_dict()

In [56]:
result['id']

'fannie_part_0_c40'

In [57]:
result['embedding'][0:5]

[0.027678849175572395,
 -0.007303171791136265,
 0.03341332823038101,
 0.06296615302562714,
 -0.014549952000379562]

In [58]:
result['gse']

'fannie'